In [0]:
import sys
import time
import random
import numpy as np
from copy import deepcopy
import pandas as pd
import matplotlib.pyplot as plt
import heapq
from itertools import combinations 

## Clase individuo

In [0]:
class Individual(object):   
   
    def __init__(self, chromosome, size):
            self.chromosome = chromosome[:]
            self.allele_pool = allele_pool
            self.fitness = -1  # -1 indica que el individuo no ha sido evaluado
            self.fitness_sep = [-1,-1,-1,-1]
            
    def crossover_onepoint(self, other):
        "Retorna dos nuevos individuos del cruzamiento de un punto entre individuos self y other "
        c = random.randrange(len(self.chromosome))
        ind1 = Individual(self.chromosome[:c] + other.chromosome[c:], allele_pool)
        ind2 = Individual(other.chromosome[:c] + self.chromosome[c:], allele_pool)
        return [ind1, ind2]   
    
    
    def crossover_uniform(self, other):
        chromosome1 = []
        chromosome2 = []
        "Retorna dos nuevos individuos del cruzamiento uniforme entre self y other "
        for i in range(len(self.chromosome)):
            if random.uniform(0, 1) < 0.5:
                chromosome1.append(self.chromosome[i])
                chromosome2.append(other.chromosome[i])
            else:
                chromosome1.append(other.chromosome[i])
                chromosome2.append(self.chromosome[i])
        ind1 = Individual(chromosome1, allele_pool)
        ind2 = Individual(chromosome2, allele_pool)
        return [ind1, ind2] 

    def mutate_position(self): 
        """       Bit flip
        Cambia aleatoriamente un alelo de un gen."""
        mutated_chromosome = deepcopy(self.chromosome)
        mutGene = random.randrange(0,len(mutated_chromosome)) 
        newAllele = allele_pool[random.randrange(0,len(allele_pool))]
        mutated_chromosome[mutGene] = newAllele
        return Individual(mutated_chromosome, allele_pool)
        
    def mutate_swap(self):
        "Escoge dos genes e intercambia sus alelos"
        mutated_chromosome = deepcopy(self.chromosome)
        mutGen1 = random.randrange(0,len(mutated_chromosome))
        mutGen2 = random.randrange(0,len(mutated_chromosome))
        temp = mutated_chromosome[mutGen1]
        mutated_chromosome[mutGen1] = mutated_chromosome[mutGen2]
        mutated_chromosome[mutGen2] = temp
        return Individual(mutated_chromosome, allele_pool)
                   

## Inicia una poblacion tipo Binario

In [0]:
def init_population(pop_number, chromosome_size, allele_pool):    
    num_alleles = len(allele_pool)
    population = []
    for i in range(pop_number):
        new_chromosome = [allele_pool[random.randrange(0, num_alleles)] for j in range(chromosome_size)]
        population.append(Individual(new_chromosome, allele_pool) )
    return population

 Convierte un Nro binario a un número natural:  
 El Nro binario es una lista de 0's y 1's 

In [0]:
def binario_a_decimal(x):
    return int(str(''.join(map(str,x))))#, 2

Funcion de aptitud para evaluar el fitness de un Cromosoma (x^2)

In [0]:
def fun_fitness_cuad(cromosoma):
    """Función que eleva al cuadrado el número recibido en binario"""
    n = 0
    for c in cromosoma:
      n +=  pob_pool[c]

    comb = combinations(cromosoma, 2) 
    dist_x = 0
    dist_y = 0
    for com in comb:
      dist_x += abs(x_pool[com[0]] - x_pool[com[1]])
      dist_y += abs(y_pool[com[0]] - y_pool[com[1]])
    
    total = dist_x + dist_y + n

    return total

In [0]:
def fun_fitness_cuad_sep(cromosoma):
    """Función que eleva al cuadrado el número recibido en binario"""
    n = 0
    for c in cromosoma:
      n +=  pob_pool[c]

    comb = combinations(cromosoma, 2) 
    dist_x = 0
    dist_y = 0
    for com in comb:
      dist_x += abs(x_pool[com[0]] - x_pool[com[1]])
      dist_y += abs(y_pool[com[0]] - y_pool[com[1]])
    
    total = dist_x + dist_y + n

    return [total,dist_x, dist_y,n]

Funcion para evaluar toda una población de individuos con la funcion de fitnes especificada

In [0]:
def evaluate_population(population,fitness_fn): 
    """Retorna el fitness de un cromosoma como el cuadrado del numero recibido en binario"""
   
    for i in range(len(population)):
        if population[i].fitness == -1:    # evalua solo si el individuo no esta evaluado
            population[i].fitness = fitness_fn(population[i].chromosome)
            population[i].fitness_sep = fun_fitness_cuad_sep(population[i].chromosome)
    return population

In [0]:
def display(population):
    listaAG=[]
    for i in range(len(population)):
      listaAG.append([population[i].chromosome,population[i].fitness,population[i].fitness_sep[1],population[i].fitness_sep[2],population[i].fitness_sep[3]])

    data=pd.DataFrame(listaAG)
    data.columns = ['Poblacion','fitness acumulado', 'Distancias X', 'Distancias Y', 'Población Acumulada']
    return data

## Selecciona los padres mediante operadores: Ruleta/ Torneo

###  Ruleta

In [0]:
def select_parents_roulette(population):
    popsize = len(population)
    
    # Escoje el primer padre
    sumfitness = sum([indiv.fitness for indiv in population])  # suma total del fitness de la poblacion
    pickfitness = random.uniform(0, sumfitness)   # escoge un numero aleatorio entre 0 y sumfitness
    cumfitness = 0     # fitness acumulado
    for i in range(popsize):
        cumfitness += population[i].fitness
        if cumfitness > pickfitness: 
            iParent1 = i
            break
     
    # Escoje el segundo padre, desconsiderando el primer padre
    sumfitness = sumfitness - population[iParent1].fitness # retira el fitness del padre ya escogido
    pickfitness = random.uniform(0, sumfitness)   # escoge un numero aleatorio entre 0 y sumfitness
    cumfitness = 0     # fitness acumulado
    for i in range(popsize):
        if i == iParent1: continue   # si es el primer padre 
        cumfitness += population[i].fitness
        if cumfitness > pickfitness: 
            iParent2 = i
            break        
    return (population[iParent1], population[iParent2])

### Torneo

In [0]:
def select_parent_torneo(population,size_torneo):
    
    # Escoje el primer padre
    list_indiv=[]
    x1 = np.random.permutation(len(population) )
    y1= x1[0:size_torneo]
    for i in range(size_torneo):
        list_indiv.append(population[y1[i]].fitness)
    
    iParent1=np.argmax(list_indiv)
    
    # Escoje el segundo padre, desconsiderando el primer padre   
    x2 = np.delete(x1, iParent1)
    x2 = np.random.permutation(x2)
    list_indiv=[]
    y2= x2[0:size_torneo]
    for i in range(size_torneo):
        list_indiv.append(population[y2[i]].fitness)
    iParent2=np.argmax(list_indiv)
    
    return (population[x1[iParent1]],population[x2[iParent2]])

## Iniciando una poblacion

### Variables

In [0]:
x_pool = [-12.04861,-12.05956,-12.06524,-12.05998,-12.05541,-12.05417,-12.05349,-12.05268,-12.05278,-12.05301,-12.04937,-12.05465,-12.05486,-12.05059,-12.04253,-12.04817,-12.04668,-12.05606,-12.07258,-12.07414,-12.06771,-12.06921,-12.07396,-12.075,-12.09086,-12.09375,-12.09602,-12.02669,-12.0282,-12.03267,-12.03828,-12.03982,-12.04117,-12.0417,-12.04155,-12.03882,-12.03959,-12.03548,-12.03766,-12.03984,-12.04233,-12.04328,-12.04486,-12.04566,-12.03895,-12.12916,-12.13156,-12.13222,-12.1265,-12.1261,-12.12603,-12.12587,-12.12248,-12.12326,-12.12028,-12.1201,-12.12532,-12.12116,-12.12159,-12.11685]
y_pool = [-77.06496,-77.07583,-77.07334,-77.063,-77.07497,-77.07472,-77.07362,-77.07409,-77.07104,-77.07735,-77.07948,-77.06952,-77.07655,-77.07666,-77.09595,-77.05867,-77.0726,-77.06726,-77.06884,-77.0629,-77.06214,-77.0593,-77.051,-77.04631,-77.05947,-77.06947,-77.07398,-76.88958,-76.8912,-76.91418,-76.9157,-76.91897,-76.9186,-76.9221,-76.9213,-76.9238,-76.92422,-76.92502,-76.92863,-76.92827,-76.92739,-76.92851,-76.92715,-76.91841,-76.92025,-77.02984,-77.02371,-77.02358,-77.02697,-77.02889,-77.03129,-77.03583,-77.03332,-77.03639,-77.03866,-77.0343,-77.03135,-77.02889,-77.03092,-77.02991]
pob_pool = [16500,18000,17000,15000,17100,17350,17300,17400,17350,17400,16900,17200,17300,17500,18500,17100,16400,15950,16800,16950,14500,16400,17300,17000,14900,13900,11500,2900,2500,2000,3800,8500,7500,6000,6000,8600,8500,9200,9900,9900,9200,8700,7900,4000,8600,16000,15800,15600,18400,18800,18300,14500,17900,16950,17600,19000,18600,18000,18500,20000]

### Codigo

In [0]:
num_individuals = 100
ind_size        = 10


allele_pool = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59]

population = init_population(num_individuals,ind_size, allele_pool)

display(population) #Imprime la primera poblacion 


,Poblacion,fitness acumulado,Distancias X,Distancias Y,Población Acumulada
0,"[31, 37, 56, 46, 2, 54, 13, 3, 30, 52]",-1,-1,-1,-1
1,"[45, 25, 6, 58, 4, 7, 46, 49, 38, 58]",-1,-1,-1,-1
2,"[29, 18, 7, 3, 12, 58, 3, 20, 26, 29]",-1,-1,-1,-1
3,"[46, 39, 52, 25, 14, 25, 15, 54, 51, 20]",-1,-1,-1,-1
4,"[1, 17, 14, 10, 22, 14, 15, 52, 43, 9]",-1,-1,-1,-1
...,...,...,...,...,...
95,"[0, 47, 53, 55, 33, 40, 41, 7, 20, 35]",-1,-1,-1,-1
96,"[29, 46, 3, 56, 44, 39, 22, 29, 43, 50]",-1,-1,-1,-1
97,"[32, 18, 21, 16, 20, 10, 6, 12, 23, 24]",-1,-1,-1,-1
98,"[9, 3, 46, 20, 34, 40, 10, 32, 33, 58]",-1,-1,-1,-1


## Evaluacion de la primera poblacion

In [0]:
fitness_fn=fun_fitness_cuad
## Evalua una poblacion
population=evaluate_population(population,fitness_fn)  # evalua la poblacion inicial
display(population)

,Poblacion,fitness acumulado,Distancias X,Distancias Y,Población Acumulada
0,"[31, 37, 56, 46, 2, 54, 13, 3, 30, 52]",140905.36758,2.10163,3.26595,140900
1,"[45, 25, 6, 58, 4, 7, 46, 49, 38, 58]",163203.98215,1.96989,2.01226,163200
2,"[29, 18, 7, 3, 12, 58, 3, 20, 26, 29]",130004.19788,1.38178,2.81610,130000
3,"[46, 39, 52, 25, 14, 25, 15, 54, 51, 20]",153604.04725,1.93812,2.10913,153600
4,"[1, 17, 14, 10, 22, 14, 15, 52, 43, 9]",161553.27965,1.03637,2.24328,161550
...,...,...,...,...,...
95,"[0, 47, 53, 55, 33, 40, 41, 7, 20, 35]",132455.24187,1.87773,3.36414,132450
96,"[29, 46, 3, 56, 44, 39, 22, 29, 43, 50]",111505.38244,2.09206,3.29038,111500
97,"[32, 18, 21, 16, 20, 10, 6, 12, 23, 24]",155002.59880,0.82151,1.77729,155000
98,"[9, 3, 46, 20, 34, 40, 10, 32, 33, 58]",126805.21666,1.56043,3.65623,126800


 ## Probando los Operadores

- Llamando a torneo

In [0]:
size_torneo = 3

winner1,winner2=select_parent_torneo(population,size_torneo)
print(winner1.chromosome,winner1.fitness)
print(winner2.chromosome,winner2.fitness)


[46, 39, 52, 25, 14, 25, 15, 54, 51, 20] 153604.04725
[59, 52, 0, 22, 43, 25, 57, 26, 58, 14] 156103.97584


- Llamando a ruleta

In [0]:
winner1,winner2=select_parents_roulette(population)
print(winner1.chromosome,winner1.fitness)
print(winner2.chromosome,winner2.fitness)

[1, 14, 55, 36, 21, 24, 34, 46, 52, 52] 152904.8881
[0, 40, 22, 8, 54, 25, 11, 11, 41, 34] 140954.39591


## Probando Cruzamiento

In [0]:
popsize = len(population)
## Selecciona las parejas de padres para cruzamiento 
mating_pool = []
for i in range(int(popsize/2)): mating_pool.append(select_parents_roulette(population)) 

In [0]:
# Crea la poblacion descendencia cruzando las parejas del mating pool con Recombinación de 1 punto
offspring_population = []
for i in range(len(mating_pool)): 
    offspring_population.extend( mating_pool[i][0].crossover_onepoint(mating_pool[i][1]) )
    #offspring_population.extend( mating_pool[i][0].crossover_uniform(mating_pool[i][1]) )

In [0]:
display(offspring_population)

,Poblacion,fitness acumulado,Distancias X,Distancias Y,Población Acumulada
0,"[39, 11, 47, 55, 9, 5, 46, 29, 28, 40]",-1,-1,-1,-1
1,"[47, 58, 58, 15, 25, 20, 7, 48, 42, 11]",-1,-1,-1,-1
2,"[16, 42, 14, 12, 55, 55, 24, 42, 55, 24]",-1,-1,-1,-1
3,"[48, 39, 4, 44, 26, 21, 22, 45, 26, 4]",-1,-1,-1,-1
4,"[59, 18, 37, 16, 25, 29, 44, 7, 47, 46]",-1,-1,-1,-1
...,...,...,...,...,...
95,"[18, 1, 59, 55, 28, 35, 57, 49, 18, 12]",-1,-1,-1,-1
96,"[55, 2, 50, 34, 48, 43, 2, 45, 6, 48]",-1,-1,-1,-1
97,"[57, 9, 54, 50, 52, 4, 6, 22, 6, 19]",-1,-1,-1,-1
98,"[28, 20, 46, 38, 43, 12, 13, 34, 30, 3]",-1,-1,-1,-1


## Probando Mutacion

In [0]:
pmut=0.8
## Aplica el operador de mutacion con probabilidad pmut en cada hijo generado
for i in range(len(offspring_population)):
    if random.uniform(0, 1) < pmut: 
        offspring_population[i] = offspring_population[i].mutate_swap()
       #offspring_population[i] = offspring_population[i].mutate_position()

In [0]:
display(offspring_population)

,Poblacion,fitness acumulado,Distancias X,Distancias Y,Población Acumulada
0,"[39, 11, 40, 55, 9, 5, 46, 29, 28, 47]",-1,-1,-1,-1
1,"[47, 58, 58, 15, 25, 20, 7, 48, 42, 11]",-1,-1,-1,-1
2,"[16, 42, 14, 12, 55, 55, 24, 42, 55, 24]",-1,-1,-1,-1
3,"[22, 39, 4, 44, 26, 21, 48, 45, 26, 4]",-1,-1,-1,-1
4,"[59, 18, 37, 16, 25, 29, 47, 7, 44, 46]",-1,-1,-1,-1
...,...,...,...,...,...
95,"[18, 1, 59, 55, 28, 35, 57, 49, 18, 12]",-1,-1,-1,-1
96,"[55, 2, 50, 34, 48, 6, 2, 45, 43, 48]",-1,-1,-1,-1
97,"[6, 9, 54, 50, 52, 4, 57, 22, 6, 19]",-1,-1,-1,-1
98,"[28, 20, 46, 38, 43, 30, 13, 34, 12, 3]",-1,-1,-1,-1


## Evalua la nueva poblacion despues de pasar por los operadores de reproduccion

In [0]:
fitness_fn=fun_fitness_cuad
## Evalua la nueva poblacion
population2=evaluate_population(offspring_population,fitness_fn)  # evalua la poblacion inicial
display(population2)

,Poblacion,fitness acumulado,Distancias X,Distancias Y,Población Acumulada
0,"[39, 11, 40, 55, 9, 5, 46, 29, 28, 47]",125955.89583,2.06783,3.82800,125950
1,"[47, 58, 58, 15, 25, 20, 7, 48, 42, 11]",159003.88391,1.90596,1.97795,159000
2,"[16, 42, 14, 12, 55, 55, 24, 42, 55, 24]",154804.62673,1.76955,2.85718,154800
3,"[22, 39, 4, 44, 26, 21, 48, 45, 26, 4]",143804.54329,1.74814,2.79515,143800
4,"[59, 18, 37, 16, 25, 29, 47, 7, 44, 46]",135705.48899,2.11912,3.36987,135700
...,...,...,...,...,...
95,"[18, 1, 59, 55, 28, 35, 57, 49, 18, 12]",155805.01190,1.95555,3.05635,155800
96,"[55, 2, 50, 34, 48, 6, 2, 45, 43, 48]",151404.67311,1.95665,2.71646,151400
97,"[6, 9, 54, 50, 52, 4, 57, 22, 6, 19]",175152.73343,1.67325,1.06018,175150
98,"[28, 20, 46, 38, 43, 30, 13, 34, 12, 3]",106305.32959,1.29395,4.03564,106300


## Juntando todo

In [0]:
print('Ejecución\tIndividuo\tIteración\tFitness\tSuma de distancias X\tSuma de distancias Y\tSuma de población')
for ejecucion in range(0,10):
  population_iter = []
  population_best = []
  best_fitness = 0
  best_a = 0
  random.seed()
  population = init_population(num_individuals,ind_size, allele_pool)
  for a in range(0,500):
    offspring_population = []
    for i in range(len(mating_pool)): 
      offspring_population.extend( mating_pool[i][0].crossover_onepoint(mating_pool[i][1]))
    for i in range(len(offspring_population)):
      if random.uniform(0, 1) < pmut: 
          offspring_population[i] = offspring_population[i].mutate_swap()
    population_iter=evaluate_population(offspring_population,fitness_fn)
    max_fitness = max(map(lambda x: x.fitness, population_iter))
    i = 0
    for pop in population_iter:
      if pop.fitness == max_fitness:
        break
      else:
        i += 1
    if population_iter[i].fitness > best_fitness:
      best_fitness = population_iter[i].fitness
      population_best = population_iter[i]
      best_a = a
  population_best.chromosome.sort()
  print(ejecucion + 1,population_best.chromosome,best_a+1,population_best.fitness,population_best.fitness_sep[1],population_best.fitness_sep[2],population_best.fitness_sep[3], sep='\t' )


Ejecución	Individuo	Iteración	Fitness	Suma de distancias X	Suma de distancias Y	Suma de población
1	[0, 1, 6, 23, 46, 49, 49, 54, 56, 59]	4	178402.76096	1.741909999999999	1.0190500000000071	178400
2	[0, 1, 6, 23, 46, 49, 49, 54, 56, 59]	5	178402.76096	1.741909999999999	1.0190500000000071	178400
3	[0, 1, 6, 23, 46, 49, 49, 54, 56, 59]	8	178402.76096	1.741909999999999	1.0190500000000071	178400
4	[0, 1, 6, 23, 46, 49, 49, 54, 56, 59]	3	178402.76096	1.741909999999999	1.0190500000000071	178400
5	[0, 1, 6, 23, 46, 49, 49, 54, 56, 59]	5	178402.76096	1.741909999999999	1.0190500000000071	178400
6	[0, 1, 6, 23, 46, 49, 49, 54, 56, 59]	11	178402.76096	1.741909999999999	1.0190500000000071	178400
7	[0, 1, 6, 23, 46, 49, 49, 54, 56, 59]	1	178402.76096	1.741909999999999	1.0190500000000071	178400
8	[0, 1, 6, 23, 46, 49, 49, 54, 56, 59]	34	178402.76096	1.741909999999999	1.0190500000000071	178400
9	[0, 1, 6, 23, 46, 49, 49, 54, 56, 59]	4	178402.76096	1.741909999999999	1.0190500000000071	178400
10	[0, 1,